In [1]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./b.part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['JOIN_MPN', 'Part_Type', 'Part_Key']]
input_.drop_duplicates(['JOIN_MPN'],
                       keep='first',
                       ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
input_['JOIN_MPN'].tolist()

总数量：2


['ROCKAUTO;GIFTCERT75', 'ROCKAUTO;T10']

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
import requests
import json

In [5]:
import sys
sys.path.append('../../00.module')
import UA
import Proxy

i = 1

jsn = '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[i, 'Part_Type'] + '","partkey":"' + input_.loc[i, 'Part_Key'] + '"}}}'

payload = {'func': 'getbuyersguide',
           'payload': jsn,
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload,
                     headers=UA.get_User_Agent_Requests(),
                     proxies=Proxy.get_Proxy_Requests()).text

resp = json.loads(resp)

resp

{'buyersguidepieces': {'body': 'No applications found.',
  'title': "Buyer's Guide :   T-Shirt"},
 'groupindex': None,
 'collected_javascript': '/**UQ:D903A642**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL809YNOHQDmNBznFtdYfbcEUriQ0mcmy\\/TcWfcbfHTjxSwndfe2chgzy88liHO2MoAcp7wW7lek5DxawTLSg\\/DpYUwwgKwrMfy30pLhGmSv8wvNozQ6rXq2UJQYJ7mj1IX7g=";\n'}

In [6]:
from bs4 import BeautifulSoup

In [7]:
html = resp['buyersguidepieces']['body']

soup = BeautifulSoup(html, 'lxml')

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <p>
   No applications found.
  </p>
 </body>
</html>



In [8]:
from lxml import etree

In [9]:
html = etree.HTML(str(soup))

list_vehicle = []
list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
for tr in list_tr:
    list_td = [text.strip() for text in tr.xpath('./td/text()')]
    list_vehicle.append(' '.join(list_td).strip())

vehicle = '\n'.join(list_vehicle)

print(vehicle)

In [10]:
crawler_status = 'ok'

print(crawler_status)

ok


In [11]:
if crawler_status == 'ok':
    df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                             'Vehicle': vehicle}])
    
    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
else:
    df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                             'Part_Type': input_.loc[i, 'Part_Type'],
                             'Part_Key': input_.loc[i, 'Part_Key']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
print(len(output_correct), len(output_error))
output_correct

1 0


,JOIN_MPN,Vehicle
0,ROCKAUTO;T10,
